# Model

We suppose that the default model of the each member is given by:

\begin{equation*}
\mathbb{P}(\tau_i \leq t) = 1 - \exp \left( - \sum_{i=0}^{k} \lambda_i (T_{i+1}-T_{i}) - \lambda_k (t-T_k) \right)
\end{equation*}

for $t \in [T_k, T_{k+1}[$, with the convention that $T_0 = 0$.

# Load matlab file

In [1]:
import scipy.io

mat = scipy.io.loadmat('Calibrated_Common_Shock_with_lambda_ind.mat')
print mat.keys()

print
print mat['__header__']
print mat['__version__']
print mat['__globals__']

['Calibrated_Common_Choc', '__version__', '__header__', '__globals__']

MATLAB 5.0 MAT-file, Platform: MACI64, Created on: Tue Apr  7 14:26:40 2015
1.0
[]


In [2]:
%matplotlib inline
import matplotlib.pyplot as plt

In [3]:
import numpy as np

calib_comm_shocks = mat['Calibrated_Common_Choc'][0, 0]
calib_comm_shocks.dtype

dtype([('Names', 'O'), ('Sectors', 'O'), ('Currencies', 'O'), ('CDS_Pillars', 'O'), ('Today', 'O'), ('coupDate', 'O'), ('CDS_Spreads', 'O'), ('CDO_attach', 'O'), ('CDO_detach', 'O'), ('CDO_Spreads', 'O'), ('CDO_Maturity', 'O'), ('Eta', 'O'), ('CDS_Pillars_in_years', 'O'), ('Common_Choc_Pillars', 'O'), ('NbNames', 'O'), ('Recovery', 'O'), ('IR', 'O'), ('Rank_group', 'O'), ('Group_size', 'O'), ('Lambda_group_optimal', 'O'), ('Lambda_ind', 'O'), ('Group_Composition', 'O')])

# Using Dataframe to get data

In [4]:
pillars = [3, 5]

## Group structure and intensities

### Group structure

In [5]:
raw_groups = calib_comm_shocks["Group_Composition"]
print raw_groups.dtype.fields

{'I_disjnt_3': (dtype('O'), 16), 'I_disjnt_2': (dtype('O'), 8), 'I_disjnt_1': (dtype('O'), 0), 'I_disjnt_5': (dtype('O'), 32), 'I_disjnt_4': (dtype('O'), 24)}


In [6]:
import re

group_composition = {int(re.findall(r'\d+', x)[0]): frozenset(raw_groups[x][0][0][0][0][0].flatten()) 
                     for x in raw_groups.dtype.fields}

In [7]:
for g in group_composition:
    print g, group_composition[g], len(group_composition[g])

1 frozenset([99, 100, 39, 105, 76, 27]) 6
2 frozenset([32, 65, 82, 71, 72, 77, 79, 112, 18, 84, 121, 125, 30]) 13
3 frozenset([67, 104, 108, 51, 56, 123]) 6
4 frozenset([1, 4, 5, 6, 10, 14, 25, 26, 28, 29, 31, 34, 37, 38, 44, 46, 48, 53, 57, 58, 62, 73, 74, 81, 83, 87, 88, 90, 91, 101, 113, 114, 115, 117, 119, 120]) 36
5 frozenset([2, 3, 7, 8, 9, 11, 12, 13, 15, 16, 17, 19, 20, 21, 22, 23, 24, 33, 35, 36, 40, 41, 42, 43, 45, 47, 49, 50, 52, 54, 55, 59, 60, 61, 63, 64, 66, 68, 69, 70, 75, 78, 80, 85, 86, 89, 92, 93, 94, 95, 96, 97, 98, 102, 103, 106, 107, 109, 110, 111, 116, 118, 122, 124]) 64


In [8]:
groups_intensities = {}

raw_group_intensities = calib_comm_shocks['Lambda_group_optimal']
for ind, pill in enumerate(pillars):
    raw_lambdas_pills = calib_comm_shocks['Lambda_group_optimal'][:, ind]
    
    cum_sum = raw_lambdas_pills.cumsum()
    groups_intensities[pill] = {i+1: c for i, c in enumerate(cum_sum)}

In [9]:
print groups_intensities[3]
print groups_intensities[5]

{1: 0.058000000000000003, 2: 0.0633745, 3: 0.063989000000000004, 4: 0.064190650000000002, 5: 0.065811350000000005}
{1: 0.015925000000000002, 2: 0.021013900000000002, 3: 0.029683100000000004, 4: 0.029776472000000005, 5: 0.037441172000000009}


## Idiosyncratic intensities

In [10]:
raw_idio = calib_comm_shocks['Lambda_ind']

I_5 = group_composition[5]

idios = []
nb_obligors = raw_idio.shape[0]

for i, idio in enumerate(raw_idio):
    id_ = i+1
    lambda_3y = idio[0] if id_ not in I_5 else 0.
    lambda_5y = idio[1] if id_ not in I_5 else 0.
    
    idios.append({"id": id_, "lambda_3y": lambda_3y, "lambda_5y": lambda_5y})

In [11]:
print "Carefull of these obligors:\n--------------------------"
for d in idios:
    if d['lambda_3y'] < 0 or d['lambda_5y'] < 0:
        print "ID = %s | l_3y = %s | l_5y = %s"%(d['id'], d['lambda_3y'], d['lambda_5y'])

Carefull of these obligors:
--------------------------
ID = 5 | l_3y = 0.0085808902955 | l_5y = -4.31167802529e-08


## Final intensities

In [12]:
groups_save = []

ids_ = frozenset()
for group_nb, ids in group_composition.iteritems():
    ids_ = ids_.union(ids, )
    print "Length for %i = %i"%(group_nb, len(ids_))
    d = {"group_id": group_nb, "compositions": ids_, 
         "lambda_3y": groups_intensities[3][group_nb],
         "lambda_5y": groups_intensities[5][group_nb]}
    
    groups_save.append(d)

Length for 1 = 6
Length for 2 = 19
Length for 3 = 25
Length for 4 = 61
Length for 5 = 125


In [13]:
idio_save = idios

In [14]:
to_save = {
    "idio": idio_save,
    "group": groups_save
}

In [15]:
import dill

In [16]:
with open('intensities.pkl', 'wb') as f:
    dill.dump(to_save, f)

In [17]:
with open('intensities.pkl', 'rb') as f:
    test = dill.load(f)